In [266]:
import redis
from redis.commands.search.query import Query
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
import pickle
import mysql.connector

In [267]:
df = pd.read_csv("data_cleaned.csv")

In [268]:
with open('scaler.pkl','rb') as f:
    sc : StandardScaler = pickle.load(f)
with open('ordinal.pkl','rb') as f:
    oe : OrdinalEncoder = pickle.load(f)

In [269]:
r = redis.Redis(
  host='localhost',
  port=6379)

In [270]:
def list_to_binary_floats(float_list):
    """Converts a list of floats into a packed binary representation."""
    return np.array(float_list, dtype=np.float32).tobytes()

In [271]:
def preprocess(query : dict):
    qq = pd.DataFrame([query])
    columns_to_categorize = ['fuel','seller_type','transmission']
    columns_to_scale = ['year', 'selling_price', 'km_driven', 'fuel','mileage','engine','max_power','seats','fuel','seller_type','transmission']
    qq[columns_to_categorize] = oe.transform(qq[columns_to_categorize])
    qq[columns_to_scale] = sc.transform(qq[columns_to_scale])
    return qq.iloc[0,:]

In [272]:
# function to perform a redis search, taking in a list of floats as input
def redis_search(query):
    
    float_list = list_to_binary_floats(preprocess(query))

    query = (
        Query("*=>[KNN 5 @vector $vec as score]")
        .sort_by("score")
        .return_fields("score")
        .paging(0, 5)
        .dialect(2)
    )

    query_params = {
        "vec": float_list
    }
    
    return r.ft("idx1").search(query, query_params).docs

In [273]:
# MySQL connection parameters
config = {
    'user': 'root',
    'password': '1234',
    'host': 'localhost',
    'database': 'veh',
    'port':'3306'
}

conn = mysql.connector.connect(**config)
cursor = conn.cursor()

In [274]:
create_table_query = """
CREATE TABLE `vehicule` (
    `id` INT(10) NULL DEFAULT NULL,
    `name` TEXT NULL DEFAULT NULL COLLATE 'utf8mb4_0900_ai_ci',
    `year` YEAR NULL DEFAULT NULL,
    `selling_price` INT(10) NULL DEFAULT NULL,
    `km_driven` INT(10) NULL DEFAULT NULL,
    `fuel` TINYTEXT NULL DEFAULT NULL COLLATE 'utf8mb4_0900_ai_ci',
    `seller_type` TINYTEXT NULL DEFAULT NULL COLLATE 'utf8mb4_0900_ai_ci',
    `transmission` TINYTEXT NULL DEFAULT NULL COLLATE 'utf8mb4_0900_ai_ci',
    `mileage` FLOAT NULL DEFAULT NULL,
    `engine` SMALLINT(5) NULL DEFAULT NULL,
    `max_power` SMALLINT(5) NULL DEFAULT NULL,
    `seats` TINYINT(3) NULL DEFAULT NULL
)
COLLATE='utf8mb4_0900_ai_ci'
ENGINE=InnoDB
;
"""
# Execute the SQL query
try:
    cursor.execute(create_table_query)
    print("Table 'vehicule' created successfully")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Error: 1050 (42S01): Table 'vehicule' already exists


In [275]:
query = \
{
    "year": 2020,
    "selling_price": 5500,
    "km_driven": 50000,
    "fuel": "Petrol",
    "seller_type": "Individual",
    "transmission": "Automatic",
    "mileage": 4,
    "engine": 2000,
    "max_power": 115.00,
    "seats": 5.0
}


In [276]:
res = redis_search(query)

In [277]:
print(res[0]["id"].split(":")[1])

5198


In [278]:
for vid in res:
    # Parameterized query to prevent SQL injection
    sql = "SELECT * FROM vehicule WHERE id = %s"
    # Assuming vid['id'].split(':')[1] results in the correct ID
    id_value = vid['id'].split(':')[1]
    
    cursor.execute(sql, (id_value,))
    
    # Fetching the result
    t = cursor.fetchone()
    
    print(t)


(5198, 'Maruti Vitara Brezza ZXI Plus AT Dual Tone', 2020, 16213, 10000, 'Petrol', 'Individual', 'Automatic', 5.33, 1462, 103, 5)
(1434, 'Tata Tiago 1.2 Revotron XZA', 2019, 12100, 30000, 'Petrol', 'Individual', 'Automatic', 4.19, 1199, 84, 5)
(2156, 'Hyundai i20 1.4 Magna AT', 2018, 15400, 42000, 'Petrol', 'Individual', 'Automatic', 5.38, 1368, 99, 5)
(4313, 'Hyundai i20 1.4 Magna AT', 2018, 14762, 20000, 'Petrol', 'Individual', 'Automatic', 5.38, 1368, 99, 5)
(1501, 'Tata Tiago 1.2 Revotron XZA', 2019, 11550, 9200, 'Petrol', 'Individual', 'Automatic', 4.19, 1199, 84, 5)


In [279]:
class DataManager:
    def __init__(self,host : str = "localhost", port : int = 6379) -> None:
        with open('scaler.pkl','rb') as f:
            self.sc : StandardScaler = pickle.load(f)
        with open('ordinal.pkl','rb') as f:
            self.oe : OrdinalEncoder = pickle.load(f)

        self.r = redis.Redis(
            host=host,
            port=port)
        
        # MySQL connection parameters
        config = {
            'user': 'root',
            'password': '1234',
            'host': 'localhost',
            'database': 'veh',
            'raise_on_warnings': True
        }

        self.conn = mysql.connector.connect(**config)
        self.cursor = self.conn.cursor()
    
    def query(self,query_dict : dict):
        res_list = self.redis_search(query_dict)
        return self.db_result_query(res_list)
    
    def preprocess(self, query : dict):
        qq = pd.DataFrame([query])
        columns_to_categorize = ['fuel','seller_type','transmission']
        columns_to_scale = ['year', 'selling_price', 'km_driven', 'fuel','mileage','engine','max_power','seats','fuel','seller_type','transmission']
        qq[columns_to_categorize] = self.oe.transform(qq[columns_to_categorize])
        qq[columns_to_scale] = self.sc.transform(qq[columns_to_scale])
        return qq.iloc[0,:]
    
    # function to perform a redis search, taking in a list of floats as input
    def redis_search(self,query : dict):
        float_list = list_to_binary_floats(self.preprocess(query))

        query = (
            Query("*=>[KNN 5 @vector $vec as score]")
            .sort_by("score")
            .return_fields("score")
            .paging(0, 5)
            .dialect(2)
        )

        query_params = {
            "vec": float_list
        }
        
        return self.r.ft("idx1").search(query, query_params).docs
        

    def db_result_query(self,redis_response_list):
        res = []
        for vid in redis_response_list:
            # Parameterized query to prevent SQL injection
            sql = "SELECT * FROM vehicule WHERE id = %s"
            id_value = vid['id'].split(':')[1]
            
            self.cursor.execute(sql, (id_value,))
            
            # Fetching the result
            res.append(self.cursor.fetchone())
        return res


In [280]:
dm = DataManager()

In [281]:
print(dm.query(query))

[(5198, 'Maruti Vitara Brezza ZXI Plus AT Dual Tone', 2020, 16213, 10000, 'Petrol', 'Individual', 'Automatic', 5.33, 1462, 103, 5), (1434, 'Tata Tiago 1.2 Revotron XZA', 2019, 12100, 30000, 'Petrol', 'Individual', 'Automatic', 4.19, 1199, 84, 5), (2156, 'Hyundai i20 1.4 Magna AT', 2018, 15400, 42000, 'Petrol', 'Individual', 'Automatic', 5.38, 1368, 99, 5), (4313, 'Hyundai i20 1.4 Magna AT', 2018, 14762, 20000, 'Petrol', 'Individual', 'Automatic', 5.38, 1368, 99, 5), (1501, 'Tata Tiago 1.2 Revotron XZA', 2019, 11550, 9200, 'Petrol', 'Individual', 'Automatic', 4.19, 1199, 84, 5)]
